In [1]:
import pandas as pd
import sqlite3

In [2]:
df = pd.read_csv("C:\\Users\\Namrata\\Downloads\\archive\\StudentsPerformance.CSV")

In [3]:
conn = sqlite3.connect(':memory:')
df.to_sql('student_performance', conn, index=False, if_exists='replace')

1000

In [4]:
print("Dataset preview:")
print(df.head())

Dataset preview:
   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  


In [5]:
def run_query(query):
    return pd.read_sql(query, conn)

In [6]:
query_1 = "SELECT * FROM student_performance WHERE gender = 'female'"
filtered_by_gender = run_query(query_1)
print("Filtered by Gender (female):")
print(filtered_by_gender)

Filtered by Gender (female):
     gender race/ethnicity parental level of education         lunch  \
0    female        group B           bachelor's degree      standard   
1    female        group C                some college      standard   
2    female        group B             master's degree      standard   
3    female        group B          associate's degree      standard   
4    female        group B                some college      standard   
..      ...            ...                         ...           ...   
513  female        group D           bachelor's degree  free/reduced   
514  female        group E             master's degree      standard   
515  female        group C                 high school  free/reduced   
516  female        group D                some college      standard   
517  female        group D                some college  free/reduced   

    test preparation course  math score  reading score  writing score  
0                      none       

In [12]:
query_2 = """
SELECT `race/ethnicity`, AVG(`math score`) as avg_math_score, AVG(`reading score`) as avg_reading_score, AVG(`writing score`) as avg_writing_score
FROM student_performance
GROUP BY `race/ethnicity`
"""
aggregated_scores = run_query(query_2)
print("Aggregated Scores by Race/Ethnicity:")
print(aggregated_scores)

Aggregated Scores by Race/Ethnicity:
  race/ethnicity  avg_math_score  avg_reading_score  avg_writing_score
0        group A       61.629213          64.674157          62.674157
1        group B       63.452632          67.352632          65.600000
2        group C       64.463950          69.103448          67.827586
3        group D       67.362595          70.030534          70.145038
4        group E       73.821429          73.028571          71.407143


In [13]:
query_3 = """
SELECT `parental level of education`, COUNT(*) as student_count
FROM student_performance
GROUP BY `parental level of education`
"""
count_by_parent_education = run_query(query_3)
print("Count by Parental Level of Education:")
print(count_by_parent_education)

Count by Parental Level of Education:
  parental level of education  student_count
0          associate's degree            222
1           bachelor's degree            118
2                 high school            196
3             master's degree             59
4                some college            226
5            some high school            179


In [15]:
query_4 = """
SELECT lunch, AVG(`math score`) as avg_math_score, AVG(`reading score`) as avg_reading_score, AVG(`writing score`) as avg_writing_score
FROM student_performance
GROUP BY lunch
"""
compare_lunch_scores = run_query(query_4)
print("Compare Scores Between Students with Standard and Free/Reduced Lunch:")
print(compare_lunch_scores)

Compare Scores Between Students with Standard and Free/Reduced Lunch:
          lunch  avg_math_score  avg_reading_score  avg_writing_score
0  free/reduced       58.921127          64.653521          63.022535
1      standard       70.034109          71.654264          70.823256


In [16]:
query_5 = """
SELECT *, (`math score` + `reading score` + `writing score`)/3.0 as average_score
FROM student_performance
ORDER BY average_score DESC
LIMIT (SELECT COUNT(*) FROM student_performance) * 0.10
"""
top_performers = run_query(query_5)
print("Top Performers (Top 10% by Average Score):")
print(top_performers)

Top Performers (Top 10% by Average Score):
    gender race/ethnicity parental level of education         lunch  \
0   female        group E           bachelor's degree      standard   
1     male        group E           bachelor's degree      standard   
2   female        group E          associate's degree      standard   
3   female        group E           bachelor's degree      standard   
4   female        group D            some high school      standard   
..     ...            ...                         ...           ...   
95    male        group D            some high school      standard   
96  female        group C           bachelor's degree      standard   
97    male        group E             master's degree      standard   
98  female        group C                some college  free/reduced   
99  female        group C          associate's degree      standard   

   test preparation course  math score  reading score  writing score  \
0                     none      

In [17]:
conn.close()